In [1]:
from dash import Dash, html, dcc, Input, Output, State
import pandas as pd
import pickle


# Caricare il file del dizionario
with open("Export\Dizionario Metriche\Minimized Version\dicMetric_Fairness_Completed.pkl", "rb") as file:
    data_dict = pickle.load(file)

# Creare il DataFrame di partenza
df = pd.DataFrame(data_dict)

# Filtrare le righe non valide (senza valori per le feature considerate)
features = ["gender", "age_group", "disability", "nationality", "education_cv_encoded"]
df_cleaned = df.dropna(subset=features, how="all")

# Creare un form Dash con i dati aggiornati
app = Dash(__name__)

# Layout dell'app
app.layout = html.Div([
    html.H1("Non-Discrimination Calculation", style={"textAlign": "center"}),

    # Tabella dinamica
    html.Div([
        html.Table([
            # Intestazione della tabella
            html.Tr([html.Th("Metric")] +
                    [html.Th(f"{feature} (Value)") for feature in features] +
                    [html.Th(f"{feature} (Impact)") for feature in features]),

            # Righe della tabella
            *[html.Tr([
                html.Td(df_cleaned.iloc[i].get("Metric", f"Metric {i+1}")),
                *[html.Td(f"{df_cleaned.iloc[i][feature]:.2f}" if pd.notna(df_cleaned.iloc[i][feature]) else "N/A",style={"textAlign": "center"}) for feature in features],
                *[html.Td(dcc.Dropdown(
                        id=f"impact-{i}-{feature}",
                        options=[{"label": v, "value": v} for v in range(11)],
                        value=0,
                        clearable=False,
                        style={"width": "60px"}
                    ),style={"textAlign": "center"}) for feature in features]
            ]) for i in range(len(df_cleaned))]
        ])
    ]),

    # Bottoni per calcolare e resettare
html.Div([
    html.Button("Calcola Metrica Globale", id="calculate-button", n_clicks=0),
    html.Button("Reset Values", id="reset-button", n_clicks=0, style={"marginLeft": "10px"}),
    html.Div(id="output-metrica", style={"fontSize": "20px", "marginLeft": "20px"})
], style={"display": "flex", "alignItems": "center", "gap": "10px"}),
])

# Callback per il calcolo della metrica globale
@app.callback(
    Output("output-metrica", "children"),
    Input("calculate-button", "n_clicks"),
    [State(f"impact-{i}-{feature}", "value") for i in range(len(df_cleaned)) for feature in features]
)
def calcola_metrica_globale(n_clicks, *impacts):
    if n_clicks == 0:
        return "Enter the impact values ​​and press the button to calculate."

    # Organizzare gli impatti
    impacts = list(impacts)
    totale_pesato = 0
    somma_impatti = 0

    for i in range(len(df_cleaned)):
        for j, feature in enumerate(features):
            valore_metrica = df_cleaned.iloc[i][feature]
            impatto = impacts[i * len(features) + j]
            if pd.notna(valore_metrica):
                totale_pesato += valore_metrica * impatto
                somma_impatti += impatto

    # Calcolo della metrica globale
    metrica_globale = totale_pesato / somma_impatti if somma_impatti > 0 else 0
    return f"Global Metric: {metrica_globale:.2f}"

# Callback per resettare i valori
@app.callback(
    [Output(f"impact-{i}-{feature}", "value") for i in range(len(df_cleaned)) for feature in features],
    Input("reset-button", "n_clicks")
)
def resetta_valori(n_clicks):
    if n_clicks > 0:
        return [0] * (len(df_cleaned) * len(features))
    from dash.exceptions import PreventUpdate
    raise PreventUpdate

# Avvio dell'app (solo se eseguito in un ambiente compatibile)
if __name__ == "__main__":
    #app.run_server(debug=True)
    app.run_server(debug=True, use_reloader=False, dev_tools_ui=True)
